In [6]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [7]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

In [8]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
mlflow.set_experiment("EXP_5-ALL_ML_hpt_7000")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/602812102375773083', creation_time=1758872187297, experiment_id='602812102375773083', last_update_time=1758872187297, lifecycle_stage='active', name='EXP_5-ALL_ML_hpt_7000', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [10]:


# Step 1: Remove rows with NaN labels
df = df.dropna(subset=['category'])

X_raw = df['clean_comment']
y_raw = df['category']

# Step 2: Train-test split first
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# Step 3: TF-IDF vectorizer fit on training data only
ngram_range = (1, 3)
max_features = 7000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply ADASYN only on training set
adasyn = ADASYN(random_state=42)
X_train_vec_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params=None):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        if best_params:
            for param, value in best_params.items():
                mlflow.log_param(param, value)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 5: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(
        y_test, 
        model.fit(X_train_vec_resampled, y_train_resampled).predict(X_test_vec)
    )

# Step 6: Run Optuna and log best model & parameters
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    best_params = study.best_params
    print("Best parameters:", best_params)
    print("Best accuracy:", study.best_value)

    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    log_mlflow(
        "SVM", 
        best_model, 
        X_train_vec_resampled, 
        X_test_vec, 
        y_train_resampled, 
        y_test, 
        best_params=best_params
    )

# Run the experiment
run_optuna_experiment()


[I 2025-09-26 14:35:00,691] A new study created in memory with name: no-name-0dc5f4a1-a974-4871-bc3c-f91c29bc4029
[I 2025-09-26 14:49:34,525] Trial 0 finished with value: 0.8112641483703805 and parameters: {'C': 1.5834542893789185, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8112641483703805.
[I 2025-09-26 14:55:35,933] Trial 1 finished with value: 0.7524887494886131 and parameters: {'C': 0.08654825728317544, 'kernel': 'linear'}. Best is trial 0 with value: 0.8112641483703805.
[I 2025-09-26 15:01:35,988] Trial 2 finished with value: 0.8246283921996455 and parameters: {'C': 0.4062091644555667, 'kernel': 'linear'}. Best is trial 2 with value: 0.8246283921996455.
[I 2025-09-26 15:07:52,059] Trial 3 finished with value: 0.7438974498840857 and parameters: {'C': 0.07778347860524397, 'kernel': 'linear'}. Best is trial 2 with value: 0.8246283921996455.
[I 2025-09-26 15:15:52,985] Trial 4 finished with value: 0.22719214509750443 and parameters: {'C': 0.017275929103256708, 'kernel': 'poly'}.

Best parameters: {'C': 1.1317205227630633, 'kernel': 'linear'}
Best accuracy: 0.8381290058639029


2025/09/26 20:02:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/26 20:02:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM_ADASYN_TFIDF_Trigrams at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083/runs/9f7cfbf4963946f298bcea7d95b1dc93
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083
